

This is a bit trivial as I already made this demo example a long time ago. I just need to tweak it to match the format of the other datasets.

In [25]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

In [26]:
from galaxy_datasets import gz_rings

ring_df, _ = gz_rings(root='/Users/user/repos/galaxy-datasets/roots/gz_rings', train=True, download=False)

In [27]:
ring_df

,id_str,brickid,objid,ring_fraction,ring_yes,ring_no,total_votes,subfolder,filename,file_loc
0,400463_1575,400463,1575,0.600000,6.0,4.0,10,400463,400463_1575.jpg,/Users/user/repos/galaxy-datasets/roots/gz_rin...
1,323205_542,323205,542,0.727273,8.0,3.0,11,323205,323205_542.jpg,/Users/user/repos/galaxy-datasets/roots/gz_rin...
2,385403_198,385403,198,0.800000,8.0,2.0,10,385403,385403_198.jpg,/Users/user/repos/galaxy-datasets/roots/gz_rin...
3,274710_2050,274710,2050,0.600000,6.0,4.0,10,274710,274710_2050.jpg,/Users/user/repos/galaxy-datasets/roots/gz_rin...
4,72670_3924,72670,3924,0.500000,5.0,5.0,10,72670,72670_3924.jpg,/Users/user/repos/galaxy-datasets/roots/gz_rin...
...,...,...,...,...,...,...,...,...,...,...
73351,236659_6421,236659,6421,0.600000,6.0,4.0,10,236659,236659_6421.jpg,/Users/user/repos/galaxy-datasets/roots/gz_rin...
73352,402717_3186,402717,3186,0.200000,2.0,8.0,10,402717,402717_3186.jpg,/Users/user/repos/galaxy-datasets/roots/gz_rin...
73353,468260_3334,468260,3334,0.695652,16.0,7.0,23,468260,468260_3334.jpg,/Users/user/repos/galaxy-datasets/roots/gz_rin...
73354,112757_2516,112757,2516,0.750000,9.0,3.0,12,112757,112757_2516.jpg,/Users/user/repos/galaxy-datasets/roots/gz_rin...


In [28]:
df = ring_df.query('total_votes >= 10').sample(1000, random_state=42)
df['ring'] = df['ring_fraction'] > 0.5

In [29]:
del df['brickid']
del df['objid']
del df['ring_yes']
del df['ring_no']
del df['total_votes']
del df['file_loc']
# delete to simplify

In [30]:
train_catalog, test_catalog = train_test_split(df, test_size=0.2, random_state=42)

In [31]:
train_catalog_loc = '/Users/user/repos/galaxy-datasets/roots/demo_rings/demo_rings_train_catalog.parquet'
test_catalog_loc = '/Users/user/repos/galaxy-datasets/roots/demo_rings/demo_rings_test_catalog.parquet'

train_catalog.to_parquet(train_catalog_loc, index=False)
test_catalog.to_parquet(test_catalog_loc, index=False)

In [32]:
# also copy the images
# will be relative to the source dir /share/nas2/walml/galaxy_zoo/decals/dr8/jpg
df['file_loc'] = df['subfolder'] + '/' + df['filename']
df[['file_loc']].to_csv('/Users/user/repos/galaxy-datasets/data/derived_data/demo_rings_file_list.csv', index=False, header=False)

# rsync -azv -

In [33]:
# e 'ssh -A -J walml@external.jb.man.ac.uk' --files-from /Users/user/repos/galaxy-datasets/data/derived_data/demo_rings_file_list.csv walml@galahad.ast.man.ac.uk:/share/nas2/walml/galaxy_zoo/decals/dr8/jpg /Users/user/repos/galaxy-datasets/data/derived_data/demo_rings

In [34]:
# rename as "images"

In [35]:
# tar czvf demo_rings_images.tar.gz images

In [37]:
import hashlib

for loc in [
    train_catalog_loc,
    test_catalog_loc,
    '/Users/user/repos/galaxy-datasets/data/derived_data/demo_rings_images.tar.gz'
]:
    # print hash
    with open(loc, 'rb') as f:
        md5_checksum = hashlib.md5(f.read()).hexdigest()

    print(md5_checksum)

4cf7e3eaab46032c374d89c349295aa2
7b28fc521836e76f794db50c3df4a2e1
5b3c6c62618bbd3165988c60e36b8365
